# Longest-Route calculation

Rule: calculate the longest route, given that we cannot pass the same station more than twice.

Special thanks to https://note.com/jnot/n/nd2c9469e4fd8

## import necessary libraries

In [1]:
pip install graphillion

     |████████████████████████████████| 1.2 MB 4.1 MB/s 
  Created wheel for graphillion: filename=Graphillion-1.5-cp37-cp37m-linux_x86_64.whl size=1729078 sha256=92f3a6f0923a9944cb216c50f4d42df08fbea3dd32331aa1134ec97a0d1cd6f7
  Stored in directory: /root/.cache/pip/wheels/94/c0/df/458922008944dec578822e0b371d7adb9699814ed95a3f8cbc
Successfully built graphillion


In [2]:
from graphillion import GraphSet as gs
import graphillion.tutorial as tl
import gc
import pandas as pd
import time
import itertools

##data import

imported as edges. The fourth column "in_use" is to define whether the given line is in operation or not.

In [3]:
#read Railway section data[s_sta,e_sta,dist,in_use]
#df_list_tmp = pd.read_csv("/path/to/Austria_Edges_v10.csv", sep=',')
df_list_tmp = pd.read_csv("/content/drive/MyDrive/2021_Vienna/Longest Route/Data/Austria_Edges_v10.csv", sep=',')

df_list = df_list_tmp[df_list_tmp["in_use"] == 1]

In [4]:
df_list

,s_sta,e_sta,dist,in_use
0,Lochau-Hoerbranz,Sud(Lauterach),8.0,1
1,Lustenau Markt,Sud(Lauterach),6.6,1
2,Sud(Lauterach),Feldkirch,32.2,1
3,Feldkirch,Tisis,7.3,1
4,Bludenz,Schruns,12.7,1
...,...,...,...,...
161,Ganserndorf,Bernhardsthal,43.9,1
162,Stockerau,Floridsdorf,21.6,1
163,Stockerau,Retz,55.6,1
164,Retz,Drosendorf,39.8,1


##create list of end/start stations

In [5]:
start_end_station_set = set.union(set(df_list['s_sta']),set(df_list['e_sta']))
print(len(start_end_station_set))
print(start_end_station_set)

df_start_end_station = pd.DataFrame(
    data={'start_set': list(start_end_station_set), 
          'end_set': list(start_end_station_set)
    }
) 

141
{'Zelamsee', 'Hainfeld', 'Pamhagen', 'Burmoos', 'Voecklamarkt', 'Alt-nagelberg', 'Arnoldstein', 'Ebenfurth', 'Emmersdorf', 'Stainach-irdning', 'Bischofshofen', 'Bad-Fischau-brunn', 'Laa-thaya', 'Waidhofen-pastalozzi', 'Weisenbach-Neuhaus', 'Wien-FJB', 'Jenbach', 'Villach', 'Grunau-im-almtal', 'Gutenstein', 'Parndorf-Ort', 'Schrambach', 'Woergl', 'Attersee', 'Scheibbs', 'Kufstein', 'Braunau-am-inn', 'Weiz', 'Unterretzbach', 'Waidhofen', 'Neusiedl-am-see', 'Heidenreichstein', 'Krimml', 'Kammer-schoerfling', 'Lieboch', 'St.Michael', 'Enns', 'Poechlarn', 'Bruck-an-der-mur', 'Drosendorf', 'Innsbruck', 'Feldkirch', 'Flughafen', 'Leopoldau', 'Stockerau', 'Unzmarkt', 'Brenner', 'Lamprechtshausen', 'Bad-Gleichenberg', 'Wernstein', 'Ganserndorf', 'Niederspaching', 'Feldbach', 'Koeschach-Mauthen', 'Wetmannstaetten', 'Marchegg', 'St.pantaleon', 'St.michael', 'Mariazell', 'Lambach', 'Baumgarten', 'Tullun', 'Handelskai', 'Summerau', 'Fehring', 'Graz', 'Leobersdorf', 'Traisen', 'Herzogenburg', 'T

## create graph

In [6]:
univ = []
weights = {}

for i, v in df_list.iterrows():
   edge = (v['s_sta'], v['e_sta'])
   univ.append(edge)
   weights[edge] = v['dist']

gs.set_universe(univ)

##Main calculation 

In [7]:
AAA = []
BBB = []

#define start and end points
for i, v in df_start_end_station.iterrows():
   sta_s=(v['start_set'])
   sta_e=(v['end_set'])
   AAA.append(sta_s)
   BBB.append(sta_e)


 #function to calculate and print the distance 
def route_calc(start,end):
  paths = gs.paths(start, end) #this single sentence can calculate all the paths btwn start and end (distance-based, decend)
  try:
    for path in paths.max_iter(weights): #extract the longest one
      max_path = path
      #print(path)
      break
    distance = 0
    for edge in max_path:
      distance += weights[edge]
    return distance

  except: 
    "StopIteration"
    return "NA"

#print the longest distance for all O-D pairs 
i_dst=0.0
j=1
time_sta=time.perf_counter()
longest_temp = 0
for A in AAA:
 for B in BBB:
   start=A
   end = B
   if(start != end ):
     i=route_calc(start,end)
     time_end=time.perf_counter()
     tim=time_end - time_sta
     if i != "NA":
       if i >= longest_temp:
        print("Calculation "+str(j)+", "+start+" to "+end+", Distance: ,"+str(i)+", elapsed time (s): "+str(tim))
        longest_temp = i
     else:
       print("Calculation "+str(j)+", "+start+" to "+end+", calculation failed,"+str(tim))

     j=j+1
 BBB.pop(0)

Calculation 1, Zelamsee to Hainfeld, Distance: ,1254.1, elapsed time (s): 0.02107507800002395
Calculation 2, Zelamsee to Pamhagen, Distance: ,1421.7999999999997, elapsed time (s): 0.03535143300001664
Calculation 3, Zelamsee to Burmoos, Distance: ,2086.2999999999997, elapsed time (s): 0.04909956700001317
Calculation 18, Zelamsee to Grunau-im-almtal, Distance: ,2118.7, elapsed time (s): 0.26221496900001284
Calculation 430, Burmoos to Jenbach, Distance: ,2204.2, elapsed time (s): 6.442867963000026
Calculation 454, Burmoos to Innsbruck, Distance: ,2238.5999999999995, elapsed time (s): 6.828608722000013
Calculation 455, Burmoos to Feldkirch, Distance: ,2395.7000000000003, elapsed time (s): 6.845691795000022
Calculation 503, Burmoos to Lustenau Markt, Distance: ,2434.5, elapsed time (s): 7.74546594200001
Calculation 509, Burmoos to Lochau-Hoerbranz, Distance: ,2435.9, elapsed time (s): 7.855634530000003
Calculation 2438, Grunau-im-almtal to Lustenau Markt, Distance: ,2466.9, elapsed time (s)

The potential longest route is now known. However, the nodes in this graph may not necessarily the start/end points. Consider the network below. In such case, the longest route is not A-B-C-D; it's A-B-C-D-(one station before)B. We call it type-P as the graph looks capital letter P. Therefore, considering that the maximum edge length is approx. 100km, we need to write down the paths longer than 2350km. These all are the potential path, and we need to (manually) consider whether there exist any type-P solutions. 

#write down all the routes over 2.350 km

In [11]:
define_long_thres = 2350

AAA = []
BBB = []

#define start and end points
for i, v in df_start_end_station.iterrows():
   sta_s=(v['start_set'])
   sta_e=(v['end_set'])
   AAA.append(sta_s)
   BBB.append(sta_e)
j = 1
for A in AAA:
 for B in BBB:
   start=A
   end = B
   if(start != end ):
     i=route_calc(start,end)
     time_end=time.perf_counter()
     tim=time_end - time_sta
     if i != "NA":
       if i >= define_long_thres:
         print("Calculation "+str(j)+", "+start+" to "+end+", Distance: ,"+str(i)+", elapsed time (s): "+str(tim))
     j=j+1
 BBB.pop(0)
 

Calculation 455, Burmoos to Feldkirch, Distance: ,2395.7000000000003, elapsed time (s): 959.879147633
Calculation 492, Burmoos to Schruns, Distance: ,2387.6000000000004, elapsed time (s): 960.402672485
Calculation 503, Burmoos to Lustenau Markt, Distance: ,2434.5, elapsed time (s): 960.5665097630001
Calculation 509, Burmoos to Lochau-Hoerbranz, Distance: ,2435.9, elapsed time (s): 960.6499468920001
Calculation 512, Burmoos to Sud(Lauterach), Distance: ,2427.9, elapsed time (s): 960.695768631
Calculation 533, Burmoos to Bludenz, Distance: ,2374.9, elapsed time (s): 960.983158989
Calculation 541, Burmoos to Tisis, Distance: ,2403.0, elapsed time (s): 961.097299522
Calculation 591, Voecklamarkt to Feldkirch, Distance: ,2375.4, elapsed time (s): 961.8207974540001
Calculation 628, Voecklamarkt to Schruns, Distance: ,2367.3, elapsed time (s): 962.3484274960001
Calculation 639, Voecklamarkt to Lustenau Markt, Distance: ,2414.2, elapsed time (s): 962.5057302770001
Calculation 645, Voecklamarkt

Luckily, in this case, we couldn't find type-P patterns which may become longer.

#Extract the longest route

##Show the order of edges

The distance is now calculated, but the above-mentioned function does not return information about the order of edges.

###Write down all the edges in the longest path.

In [9]:
start_sta = "Lochau-Hoerbranz"
goal_sta = "Grunau-im-almtal"
paths = gs.paths(start_sta, goal_sta)
for path in paths.max_iter(weights):
  max_path = path
  print(path)
  break

[('Absdorf-hippersdorf', 'Stockerau'), ('Amstetten', 'Poechlarn'), ('Amstetten', 'St.Valentin'), ('Attnang-Puchheim', 'Stainach-irdning'), ('Bischofshofen', 'Salzburg'), ('Bischofshofen', 'Stainach-irdning'), ('Bludenz', 'Innsbruck'), ('Bruck-an-der-mur', 'Graz'), ('Ebenfurth', 'Wulkaprodersdorf'), ('Fehring', 'Wr-Neustadt'), ('Feldbach', 'Fehring'), ('Feldkirch', 'Bludenz'), ('Floridsdorf', 'Leopoldau'), ('Ganserndorf', 'Leopoldau'), ('Ganserndorf', 'Marchegg'), ('Gleisdorf', 'Feldbach'), ('Graz', 'Gleisdorf'), ('Hadersdorf', 'Sigmundsherberg'), ('Handelskai', 'Heiligenstadt'), ('Handelskai', 'Rennweg'), ('Hbf', 'Flughafen'), ('Hbf', 'Marchegg'), ('Herzogenburg', 'Krems-donau'), ('Herzogenburg', 'Tullnerfeld'), ('Hutteldorf', 'Penzing'), ('Innsbruck', 'Jenbach'), ('Jenbach', 'Woergl'), ('Kastenreith', 'St.Valentin'), ('Klagenfurt', 'Rosenbach'), ('Kledering', 'Sollenau'), ('Krems-donau', 'Hadersdorf'), ('Lambach', 'Attnang-Puchheim'), ('Lambach', 'Wels'), ('Leobersdorf', 'Meidling'), 

###Sort these edges by the appearing oreder

In [10]:
start_temp = start_sta
print(start_sta)
for i in range(len(max_path)):
  for start_end_pair in max_path:
    if start_sta == start_end_pair[0]:
      print("->",start_end_pair[1],end='')
      start_sta = start_end_pair[1]
      max_path.remove(start_end_pair)
      break
    elif start_sta == start_end_pair[1]:
      print("->",start_end_pair[0],end='')
      start_sta = start_end_pair[0]
      max_path.remove(start_end_pair)
      break

Lochau-Hoerbranz
-> Sud(Lauterach)-> Feldkirch-> Bludenz-> Innsbruck-> Jenbach-> Woergl-> Zelamsee-> Schwarzach-St.Veit-> Spittal-milstaettersee-> Villach-> VillachWarmbad-> Rosenbach-> Klagenfurt-> St.Veit-an-der-Glan-> Unzmarkt-> St.michael-> Bruck-an-der-mur-> Graz-> Gleisdorf-> Feldbach-> Fehring-> Wr-Neustadt-> Ebenfurth-> Wulkaprodersdorf-> Neusiedl-am-see-> Parndorf-Ort-> Kledering-> Sollenau-> Leobersdorf-> Meidling-> Tullnerfeld-> Herzogenburg-> Krems-donau-> Hadersdorf-> Sigmundsherberg-> Absdorf-hippersdorf-> Stockerau-> Floridsdorf-> Leopoldau-> Ganserndorf-> Marchegg-> Hbf-> Flughafen-> Rennweg-> Handelskai-> Heiligenstadt-> Penzing-> Hutteldorf-> St.polten-> Poechlarn-> Amstetten-> St.Valentin-> Kastenreith-> Selzthal-> Linz-> Niederspaching-> Neumarkt-Kallham-> Scharding-> Ried-im-innkreis-> Braunau-am-inn-> Steindorf-> Salzburg-> Bischofshofen-> Stainach-irdning-> Attnang-Puchheim-> Lambach-> Wels-> Grunau-im-almtal